In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

path_root = '../..'
sys.path.append(f'{path_root}/')
from src.util.functions import Functions as F

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = f'{path_root}/data'
path_csv = f'{path_data}/csv'

csv_out_name = f'{path_csv}/covid19-{level}.csv'
pop_regions_name = f'{path_csv}/popolazione-regioni.csv'
icu_regions_name = f'{path_csv}/terapie-intensive.csv'
regions_name = f'{path_csv}/regioni.csv'

In [2]:
url_cum = F.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,note,ingressi_terapia_intensiva,note_test,note_casi,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14170,2021-12-29T17:00:00,ITA,19,Sicilia,38.115697,13.362357,703,89,792,34436,...,DECEDUTI: N. 2 IL 28/12/21 - N. 4 IL 27/12/21,7.0,NaN,NaN,363044.0,0.0,3784217.0,4504236.0,ITG,ITG1
14171,2021-12-29T17:00:00,ITA,9,Toscana,43.769231,11.255889,549,78,627,42009,...,NaN,9.0,NaN,NaN,341621.0,7262.0,5394707.0,3598405.0,ITI,ITI1
14172,2021-12-29T17:00:00,ITA,10,Umbria,43.106758,12.388247,114,8,122,13889,...,NaN,1.0,NaN,NaN,79318.0,3602.0,1336105.0,1511990.0,ITI,ITI2
14173,2021-12-29T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,28,2,30,1556,...,NaN,0.0,NaN,NaN,13232.0,2314.0,124980.0,226174.0,ITC,ITC2


In [4]:
list(df_raw_r.columns)

['data',
 'stato',
 'codice_regione',
 'denominazione_regione',
 'lat',
 'long',
 'ricoverati_con_sintomi',
 'terapia_intensiva',
 'totale_ospedalizzati',
 'isolamento_domiciliare',
 'totale_positivi',
 'variazione_totale_positivi',
 'nuovi_positivi',
 'dimessi_guariti',
 'deceduti',
 'casi_da_sospetto_diagnostico',
 'casi_da_screening',
 'totale_casi',
 'tamponi',
 'casi_testati',
 'note',
 'ingressi_terapia_intensiva',
 'note_test',
 'note_casi',
 'totale_positivi_test_molecolare',
 'totale_positivi_test_antigenico_rapido',
 'tamponi_test_molecolare',
 'tamponi_test_antigenico_rapido',
 'codice_nuts_1',
 'codice_nuts_2']

In [5]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'casi_da_sospetto_diagnostico', 'casi_da_screening', 'note', 'note_test', 'note_casi']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,...,0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,...,0,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,...,18,148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14170,2021-12-29T17:00:00,19,Sicilia,703,89,792,34436,35228,3085,3729,...,363044,8288453,2977862.0,7.0,363044.0,0.0,3784217.0,4504236.0,ITG,ITG1
14171,2021-12-29T17:00:00,9,Toscana,549,78,627,42009,42636,6501,7304,...,348883,8993112,3606436.0,9.0,341621.0,7262.0,5394707.0,3598405.0,ITI,ITI1
14172,2021-12-29T17:00:00,10,Umbria,114,8,122,13889,14011,2920,3171,...,82920,2848095,512984.0,1.0,79318.0,3602.0,1336105.0,1511990.0,ITI,ITI2
14173,2021-12-29T17:00:00,2,Valle d'Aosta,28,2,30,1556,1586,207,229,...,15546,351154,107990.0,0.0,13232.0,2314.0,124980.0,226174.0,ITC,ITC2


In [6]:
# Compute most recent couple of dates
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = F.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2021-12-29', '2021-12-28')

In [7]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2021-12-29T17:00:00,1,Piemonte,1147,99,1246,53707,54953,7057,9671,...,471753,11152575,3027869.0,14.0,395618.0,76135.0,4124051.0,7028524.0,ITC,ITC1
1,2021-12-29T17:00:00,2,Valle d'Aosta,28,2,30,1556,1586,207,229,...,15546,351154,107990.0,0.0,13232.0,2314.0,124980.0,226174.0,ITC,ITC2
2,2021-12-29T17:00:00,3,Lombardia,1831,191,2022,176635,178657,28466,32696,...,1136754,24480765,6733019.0,4.0,942652.0,194102.0,12880683.0,11600082.0,ITC,ITC4
3,2021-12-29T17:00:00,5,Veneto,1167,180,1347,79109,80456,4485,8666,...,626319,19311397,2447628.0,27.0,576745.0,49574.0,8238558.0,11072839.0,ITH,ITH3
4,2021-12-29T17:00:00,6,Friuli Venezia Giulia,278,27,305,9463,9768,717,1817,...,151626,4472482,957398.0,3.0,131599.0,20027.0,2626586.0,1845896.0,ITH,ITH4
5,2021-12-29T17:00:00,7,Liguria,505,38,543,11608,12151,628,1634,...,144859,3495050,1036373.0,5.0,144859.0,0.0,1976219.0,1518831.0,ITC,ITC3
6,2021-12-29T17:00:00,8,Emilia-Romagna,1305,113,1418,61085,62503,3131,4134,...,519672,11548966,2380247.0,11.0,519052.0,620.0,7226848.0,4322118.0,ITH,ITH5
7,2021-12-29T17:00:00,9,Toscana,549,78,627,42009,42636,6501,7304,...,348883,8993112,3606436.0,9.0,341621.0,7262.0,5394707.0,3598405.0,ITI,ITI1
8,2021-12-29T17:00:00,10,Umbria,114,8,122,13889,14011,2920,3171,...,82920,2848095,512984.0,1.0,79318.0,3602.0,1336105.0,1511990.0,ITI,ITI2
9,2021-12-29T17:00:00,11,Marche,218,43,261,6938,7199,186,1707,...,141571,1973115,1151718.0,4.0,141571.0,0.0,1626595.0,346520.0,ITI,ITI3


In [8]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2021-12-28T17:00:00,1,Piemonte,1118,92,1210,46686,47896,5745,7933,...,462082,11071749,3002610.0,14.0,393615.0,68467.0,4108959.0,6962790.0,ITC,ITC1
1,2021-12-28T17:00:00,2,Valle d'Aosta,29,2,31,1348,1379,144,253,...,15317,348516,107176.0,0.0,13188.0,2129.0,124649.0,223867.0,ITC,ITC2
2,2021-12-28T17:00:00,3,Lombardia,1698,193,1891,148300,150191,26311,28795,...,1104058,24271080,6690646.0,14.0,931995.0,172063.0,12825164.0,11445916.0,ITC,ITC4
3,2021-12-28T17:00:00,5,Veneto,1050,165,1215,74756,75971,4210,7403,...,617653,19191330,2441330.0,25.0,572593.0,45060.0,8209702.0,10981628.0,ITH,ITH3
4,2021-12-28T17:00:00,6,Friuli Venezia Giulia,283,26,309,8742,9051,219,737,...,149812,4445465,953936.0,0.0,130117.0,19695.0,2615664.0,1829801.0,ITH,ITH4
5,2021-12-28T17:00:00,7,Liguria,508,41,549,10974,11523,990,1146,...,143225,3473714,1031890.0,3.0,143225.0,0.0,1968108.0,1505606.0,ITC,ITC3
6,2021-12-28T17:00:00,8,Emilia-Romagna,1286,112,1398,57974,59372,1653,3427,...,515549,11495602,2373631.0,14.0,514948.0,601.0,7195723.0,4299879.0,ITH,ITH5
7,2021-12-28T17:00:00,9,Toscana,520,80,600,35535,36135,3862,4453,...,341579,8937048,3591936.0,10.0,334621.0,6958.0,5370673.0,3566375.0,ITI,ITI1
8,2021-12-28T17:00:00,10,Umbria,104,8,112,10979,11091,2584,2717,...,79749,2824913,503762.0,0.0,77631.0,2118.0,1330175.0,1494738.0,ITI,ITI2
9,2021-12-28T17:00:00,11,Marche,212,42,254,6759,7013,187,1098,...,139864,1959677,1142553.0,0.0,139864.0,0.0,1618394.0,341283.0,ITI,ITI3


In [9]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = df_raw_r_1
# df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
# df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

In [10]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = df_raw_r_0
# df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
# df_r_0.sort_values(by='codice_regione').reset_index()

In [11]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione
0,2021-12-29T17:00:00,1,Piemonte,1147,99,1246,53707,54953,7057,9671,...,11152575,3027869.0,14.0,395618.0,76135.0,4124051.0,7028524.0,ITC,ITC1,4.341375e+06
1,2021-12-29T17:00:00,2,Valle d'Aosta,28,2,30,1556,1586,207,229,...,351154,107990.0,0.0,13232.0,2314.0,124980.0,226174.0,ITC,ITC2,1.255010e+05
2,2021-12-29T17:00:00,3,Lombardia,1831,191,2022,176635,178657,28466,32696,...,24480765,6733019.0,4.0,942652.0,194102.0,12880683.0,11600082.0,ITC,ITC4,1.010397e+07
3,2021-12-29T17:00:00,5,Veneto,1167,180,1347,79109,80456,4485,8666,...,19311397,2447628.0,27.0,576745.0,49574.0,8238558.0,11072839.0,ITH,ITH3,4.907704e+06
4,2021-12-29T17:00:00,6,Friuli Venezia Giulia,278,27,305,9463,9768,717,1817,...,4472482,957398.0,3.0,131599.0,20027.0,2626586.0,1845896.0,ITH,ITH4,1.211357e+06
5,2021-12-29T17:00:00,7,Liguria,505,38,543,11608,12151,628,1634,...,3495050,1036373.0,5.0,144859.0,0.0,1976219.0,1518831.0,ITC,ITC3,1.543127e+06
6,2021-12-29T17:00:00,8,Emilia-Romagna,1305,113,1418,61085,62503,3131,4134,...,11548966,2380247.0,11.0,519052.0,620.0,7226848.0,4322118.0,ITH,ITH5,4.467118e+06
7,2021-12-29T17:00:00,9,Toscana,549,78,627,42009,42636,6501,7304,...,8993112,3606436.0,9.0,341621.0,7262.0,5394707.0,3598405.0,ITI,ITI1,3.722729e+06
8,2021-12-29T17:00:00,10,Umbria,114,8,122,13889,14011,2920,3171,...,2848095,512984.0,1.0,79318.0,3602.0,1336105.0,1511990.0,ITI,ITI2,8.802850e+05
9,2021-12-29T17:00:00,11,Marche,218,43,261,6938,7199,186,1707,...,1973115,1151718.0,4.0,141571.0,0.0,1626595.0,346520.0,ITI,ITI3,1.518400e+06


In [12]:
# Update icu csv data
df_icu_updated_r = pd.DataFrame.from_dict(F.icu_data())
df_icu_updated_r.to_csv(icu_regions_name, index=False)

In [13]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, left_on='denominazione_regione', right_on='regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione,regione,ricoverati_area_non_critica,posti_letto_area_non_critica,ricoverati_terapia_intensiva,posti_letto_terapia_intensiva,posti_letto_terapia_intensiva_attivabili
0,2021-12-29T17:00:00,1,Piemonte,1147,99,1246,53707,54953,7057,9671,...,7028524.0,ITC,ITC1,4.341375e+06,Piemonte,1147,5824,99,628,99
1,2021-12-29T17:00:00,2,Valle d'Aosta,28,2,30,1556,1586,207,229,...,226174.0,ITC,ITC2,1.255010e+05,Valle d'Aosta,28,99,2,33,0
2,2021-12-29T17:00:00,3,Lombardia,1831,191,2022,176635,178657,28466,32696,...,11600082.0,ITC,ITC4,1.010397e+07,Lombardia,1831,10457,191,1530,0
3,2021-12-29T17:00:00,5,Veneto,1167,180,1347,79109,80456,4485,8666,...,11072839.0,ITH,ITH3,4.907704e+06,Veneto,1167,6000,180,1000,0
4,2021-12-29T17:00:00,6,Friuli Venezia Giulia,278,27,305,9463,9768,717,1817,...,1845896.0,ITH,ITH4,1.211357e+06,Friuli Venezia Giulia,278,1277,27,175,0
5,2021-12-29T17:00:00,7,Liguria,505,38,543,11608,12151,628,1634,...,1518831.0,ITC,ITC3,1.543127e+06,Liguria,505,1764,38,219,12
6,2021-12-29T17:00:00,8,Emilia-Romagna,1305,113,1418,61085,62503,3131,4134,...,4322118.0,ITH,ITH5,4.467118e+06,Emilia-Romagna,1305,9001,113,889,0
7,2021-12-29T17:00:00,9,Toscana,549,78,627,42009,42636,6501,7304,...,3598405.0,ITI,ITI1,3.722729e+06,Toscana,549,5033,78,570,21
8,2021-12-29T17:00:00,10,Umbria,114,8,122,13889,14011,2920,3171,...,1511990.0,ITI,ITI2,8.802850e+05,Umbria,114,662,8,86,41
9,2021-12-29T17:00:00,11,Marche,218,43,261,6938,7199,186,1707,...,346520.0,ITI,ITI3,1.518400e+06,Marche,218,980,43,252,51


In [14]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [15]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['posti_letto_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['posti_letto_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2021-12-29T17:00:00,3,Lombardia,1831,191,2022,176635,178657,28466,32696,...,12.48,28335,3901,209685,42373.0,28,4202,15.59,0.32360,11.25
1,2021-12-29T17:00:00,15,Campania,587,37,624,54166,54790,8476,9802,...,6.73,8421,2621,111379,16715.0,11,1315,8.80,0.16941,9.63
2,2021-12-29T17:00:00,1,Piemonte,1147,99,1246,53707,54953,7057,9671,...,15.76,7021,1738,80826,25259.0,6,2608,11.97,0.22276,10.87
3,2021-12-29T17:00:00,5,Veneto,1167,180,1347,79109,80456,4485,8666,...,18.00,4353,1263,120067,6298.0,23,4158,7.22,0.17658,12.76
4,2021-12-29T17:00:00,9,Toscana,549,78,627,42009,42636,6501,7304,...,13.68,6474,2851,56064,14500.0,9,794,13.03,0.19620,9.37
5,2021-12-29T17:00:00,12,Lazio,1070,147,1217,58896,60113,4050,5248,...,15.59,3993,960,88525,3755.0,10,1188,5.93,0.08947,8.37
6,2021-12-29T17:00:00,8,Emilia-Romagna,1305,113,1418,61085,62503,3131,4134,...,12.71,3111,707,53364,6616.0,11,981,7.75,0.09254,11.63
7,2021-12-29T17:00:00,19,Sicilia,703,89,792,34436,35228,3085,3729,...,10.35,3066,910,55631,16329.0,6,638,6.70,0.07505,7.31
8,2021-12-29T17:00:00,10,Umbria,114,8,122,13889,14011,2920,3171,...,9.30,2910,454,23182,9222.0,1,250,13.68,0.36022,9.42
9,2021-12-29T17:00:00,16,Puglia,203,24,227,15808,16035,2155,2885,...,4.98,2149,928,73314,9990.0,7,723,3.94,0.07198,7.46


In [16]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [17]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'posti_letto_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,posti_letto_terapia_intensiva,saturazione_terapia_intensiva
0,P.A. Trento,21,69,90,23.33
1,Veneto,180,820,1000,18.00
2,P.A. Bolzano,18,82,100,18.00
3,Liguria,38,181,219,17.35
4,Marche,43,209,252,17.06
5,Piemonte,99,529,628,15.76
6,Lazio,147,796,943,15.59
7,Friuli Venezia Giulia,27,148,175,15.43
8,Calabria,28,159,187,14.97
9,Toscana,78,492,570,13.68


In [18]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['variazione_totale_positivi'] / (df['totale_positivi'] - df['variazione_totale_positivi'])).replace(np.inf, 0) * 100).round(2)
    return df.fillna(0)

In [19]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../../data/csv/regioni/covid19-veneto.csv
Aggiornato ../../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../../data/csv/regioni/covid19-liguria.csv
Aggiornato ../../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../../data/csv/regioni/covid19-toscana.csv
Aggiornato ../../data/csv/regioni/covid19-umbria.csv
Aggiornato ../../data/csv/regioni/covid19-marche.csv
Aggiornato ../../data/csv/regioni/covid19-lazio.csv
Aggiornato ../../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../../data/csv/regioni/covid19-molise.csv
Aggiornato ../../data/csv/regioni/covid19-campania.csv
Aggiornato ../../data/csv/regioni/covid19-puglia.csv
Aggiornato ../../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../../data/csv/regioni/covid19-calabria.csv
Aggiornato ../../data/csv/regioni/covid19-sicilia.csv


In [20]:
# df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
# df_tb

In [21]:
# Initialize df_taa
# df_taa = Functions.replace_bt_with_taa(df_tb[:2])
# for i in range(2, df_tb.shape[0], 2):
#     df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
# df_taa = add_cols(df_taa)
# df_taa

In [22]:
# region_name = df_taa.denominazione_regione.iloc[0].lower()
# csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
# df_taa.to_csv(csv_out_region_name, index=False)
# print('Aggiornato {file}'.format(file=csv_out_region_name))